In [2]:
import re
import os, os.path
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy
import pandas as pd
import warnings

In [15]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [73]:
# process generated files
abs_path = r'/home/yurii/projects/mownit2/lab6/outl/AB/'
cur_path = r'/home/yurii/projects/mownit2/lab6/a/'
for filename in os.listdir(abs_path):
    with open(abs_path + filename, 'r') as myfile:
        s = myfile.read()
        p = re.compile('<doc[^>]*>([^<]*)<\/doc>')
        l = p.findall(s)
        for i in range(0,len(l)):
            p = re.compile('Notes\n|Footnotes\n|Bibliography\n|References\n|External links\n|Further reading\n')
            l[i] = p.sub(r'', l[i])
            p = re.compile('\n{3,}')
            l[i] = p.sub(r'\n', l[i])
            if l[i][0] == '\n':
                l[i] = l[i][1:]
            title = l[i].splitlines()[0]
            title = title.replace(r'/', ' ')
            with open(cur_path + title, 'w+') as myfile2:
                myfile2.write(l[i])

In [10]:
art_path = r'/home/yurii/projects/mownit2/lab6/ex/'
file_list = [name for name in os.listdir(art_path)]

In [22]:
warnings.filterwarnings("ignore", category=DeprecationWarning, module="pandas", lineno=570)

In [28]:
class PorterTokenizer(object):
    def __init__(self):
        self.pt = PorterStemmer()
    def __call__(self, doc):
        return [self.pt.stem(t) for t in RegexpTokenizer(r'(?u)\b\w\w+\b').tokenize(doc)]

In [29]:
example = [
    'In mathematics, 1 − 2 + 3 − 4 + ··· is the infinite series whose terms are the successive positive integers, given alternating signs. Using sigma summation notation the sum of the first "m" terms of the series can be expressed as',
    'The infinite series diverges, meaning that its sequence of partial sums, , does not tend towards any finite limit. Nonetheless, in the mid-18th century, Leonhard Euler wrote what he admitted to be a paradoxical equation:',
    "A rigorous explanation of this equation would not arrive until much later. Starting in 1890, Ernesto Cesàro, Émile Borel and others investigated well-defined methods to assign generalized sums to divergent series—including new interpretations of Euler's attempts."
]

vect1 = CountVectorizer(min_df=1, stop_words='english', tokenizer=PorterTokenizer())
# vect2 = CountVectorizer(min_df=1, stop_words='english')
dtm1 = vect1.fit_transform(example)
# dtm2 = vect2.fit_transform(example)

In [ ]:
# TODO: IDF

In [32]:
# pd.DataFrame(dtm1.toarray(), index=example, columns=vect1.get_feature_names()).head(10)
# vect1.get_feature_names()

In [33]:
# pd.DataFrame(dtm2.toarray(), index=example, columns=vect2.get_feature_names()).head(10)
# vect2.get_feature_names()

In [12]:
# dict_list = [col.OrderedDict([]) for i in range(0,len(file_list))]

# def read_words(words_file):
#     return [word for line in open(words_file, 'r') for word in line.split()]

# file_name = art_path + r'0'
# wl = read_words(file_name)